In [1]:
# general imports, basic globals

import io
import sys
import subprocess 
import copy
import pandas as pd
import numpy as np
import pyvista as pv

import galois
import json

# from er_viz import visualize_graph

# Define some colors, for the vertices and edges
RED = '#FF0000'
DK_RED = '#990000'
LT_GREEN = '#44FF44'
DK_GREEN = '#009900'
LT_BLUE = '#0088FF'


In [2]:
def visualize_graph(graph, layout, 
                    edgelists_to_display, 
                    camera_pos, camera_rot , 
                    show_labels, fname, caption):
# lighting
    BACKGROUND_COLOR = 'white'
    LIGHT_COLOR = 'white'
    pl = pv.Plotter(lighting=None, window_size=(2000, 2000))
    light1 = pv.Light(poasition=(10, 10., -5.0), focal_point=(0, 0, 0), color=LIGHT_COLOR, intensity=1.5)
    light2 = pv.Light(position=(-10, 10.0, -5.0), focal_point=(0, 0, 0), color=LIGHT_COLOR, intensity=0.75)
#     pl.add_light(light1)  # Lighting effect to be cast onto the object
#     pl.add_light(light2)  # Second Lighting effect to be cast onto the object
    hlight = pv.Light(light_type='headlight')
    pl.add_light(hlight)  # headlight effect to be cast onto the object
    pl.set_background(BACKGROUND_COLOR)  # Set the background     
    
    try:
        layout_num = graph.layouts.layout_names.index(layout)
    except ValueError:
        print(layout, " is not in the list of layouts.")

# vertices
# Sort by color then pl.add_mesh much faster than pl.add_points! 
# also (for now) more general than sorting by cluster set
    colorset = []
    color_vertices = []
    for vertex in graph.vertex_list:
        try:
            c = colorset.index(vertex.color)
            color_vertices[c].append(vertex.pos[layout_num])
        except ValueError:
            colorset.append(vertex.color)
            color_vertices.append([vertex.pos[layout_num]])
    for c, arg in enumerate(colorset):
        cloud = pv.PolyData(color_vertices[c])
        pl.add_mesh(cloud, 
                color=arg, 
                point_size=graph.vertex_list[0].size, # making them all the same size
                render_points_as_spheres=True
               )
# edge sets
    vpos = []
    for v in graph.vertex_list:
        vpos.append(v.pos[layout_num])
    vertices = np.array(vpos)
    for el in edgelists_to_display:
        try:
            i = graph.edges_list.names.index(el)
        except ValueError:
            print(el, " is not in the list of edgesets.")            
        num_edges = len(graph.edges_list.list[i].edges)
        elist = []
        for j in range(num_edges):
            elist.append(graph.edges_list.list[i].edges[j].edge)
        if elist:
            estack = np.hstack(elist)
            edge_mesh = pv.PolyData(vertices, lines=estack, n_lines = num_edges)
            pl.add_mesh(edge_mesh, color = graph.edges_list.list[i].color, opacity = graph.edges_list.list[i].opacity, line_width = graph.edges_list.list[i].line_width, point_size=0)        
        else:
            print('No edges in this graph')

# camera views: xy, xz, yz, yx, zx, zy, iso 
    pl.camera_position = camera_pos
    pl.camera.roll = camera_rot[0]
    pl.camera.azimuth = camera_rot[1]
    pl.camera.elevation = camera_rot[2]
    pl.camera.zoom(1)      
# screenshot
    pl.screenshot(fname, window_size=[2000,2000])
    #pl.show_axes()    
    print(caption)
    pl.show()


In [3]:
# finite field GF(q) class

class GFq:
    # finite field arithmetic tables and 3-D dot product calculation
    def __init__(self, q):
        self.q = q
        self.GF_q_add, self.GF_q_mul = self.make_GF_q_arith_table(q)
        
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def make_GF_q_arith_table(self,q):
    # addition and multiplication tables in GF(q)
        GF_q_add = []
        GF_q_mul = []
        GF = galois.GF(self.q)
        for i in range(self.q):
            x_vec = self.q*[i]
            y_vec = [j for j in range(self.q)]
            x = GF(x_vec)
            y = GF(y_vec)
            GF_q_add.append(json.loads(str(x+y)))
            GF_q_mul.append(json.loads(str(x*y)))
        return GF_q_add, GF_q_mul    

    def dot_product_3d(self,v,w):
        mul0 = self.GF_q_mul[v[0]][w[0]]
        mul1 = self.GF_q_mul[v[1]][w[1]]
        mul2 = self.GF_q_mul[v[2]][w[2]]
        return self.GF_q_add[ self.GF_q_add[mul0][mul1] ][mul2]


In [4]:
# some graph support classes

class Vertex:
    def __init__(self, value):
        self.value = value
        self.adj = []  # vertices that are adjacent to self
        self.pos = []  # a set of positions for self. self.pos[i] is self position in the ith layout
        self.color = 'black'   # what color do you want the default vertex to be
        self.size = 20   # what size do you want the default vertex to be
        
    def __getitem__(self, key):
        return self.__dict__[key]
    a
    def __str__(self):
        return str(self.value)
        
class VertexList:
    def __init__(self):
        self.list = []

    def __getitem__(self, key):
        return self.__dict__[key]

    def __str__(self):
        string = ''
        for l in self.list:
            string += str(l)
        string += '\n'
        return string

class Position: # cartesian
    def __init__self(self,x,y,z):
        self.x = x
        self.y = y
        self.z = z
        
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __str__(self):
        return str('(' + self.x +', ' + self.y +', ' + self.z +')')

class Value:  # 3-vectors
    def __init__self(self, a,b,c):
        self.v0 = a
        self.v1 = b
        self.v2 = c  
                        
    def __getitem__(self, key):
        return self.__dict__[key]

    def __str__(self):
        return str('(' + self.v0 +', ' + self.v1 +', ' + self.v2 +')')
        
class Edge:
    def __init__(self, v1, v2):
        self.edge = [2, v1, v2]
        
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __str__(self):
        return str(self.edge)


class Edgeset:
    # this is a set of edges. Color, opacity, line_width defined per Edgeset, not per Edge
    def __init__(self, color, opacity, line_width):
        self.edges = []
        self.opacity = opacity
        self.color = color
        self.line_width = line_width
        
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __str__(self):
        string = ''
        for e in self.edges:
            string += str(e)
        string += '\n'
        return string
    
    def append_edge(self, edge):
        self.edgeset.append(edge)
        
    def concatenate_edgesets(self, edgeset):
        new_edgeset = self
        for e in edgeset:
            new_edgeset.edges.append(e)
        new_edgeset.color = self.color
        new_edgeset.opacity = self.opacity
        new_edgeset.line_width = self.line_width
        return new_edgeset
            
    def num_edges(self):
        return len(self.edges)

    
class EdgesList:
    # this is a list of edge sets
    def __init__(self, edgeset, name):
        self.list = [edgeset]
        self.names = [name]
        
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __str__(self):
        string = ''
        for el in self.list:
            string += str(el)
        return string
    
    def append_edgeset(self, edgeset):
    # append a set of edges to the current edge_sets list, along with their positions and drawing details
        self.list.append(edgeset)  
        
    def num_lists(self):
        return len(self.list)


    

In [5]:
# graph base class

class Graph:

    def __init__(self, num):
        self.vertex_list, self.num_vertices = self.set_vertex_list(num) # should be set by child class 
        self.inc_matrix = self.set_incidence_matrix() # must be set by child class         
        self.set_adj_lists()  
        self.indices = [i for i in range(self.num_vertices)]
        # default for self.edges_list is a list with all of the edges of the graph
        self.edges_list = EdgesList(self.get_edges_within_indexset(self.indices, 'black', 0.2, 1), 'all_edges')
        # default for self.layouts is a circular layout with a clockwise ordering as per self.indices
        self.set_layouts()
        self.init_stuff()

    def __getitem__(self, key):
        return self.__dict__[key]
    
    def set_vertex_list(self, num):
        vertex_list = []
        for i in range(0,num):
            vertex_list.append(Vertex(i))
        return vertex_list, len(vertex_list)
        pass
    
    def set_incidence_matrix(self):
        inc_matrix = []
        for i in range(self.num_vertices):
            row = [0]*self.num_vertices
            inc_matrix.append(row)
        return inc_matrix
    
    def set_adj_lists(self):
        num_vlist = len(self.vertex_list)
        for i in range(num_vlist):
            if self.inc_matrix[i][i]==1:
                self.vertex_list[i].adj.append(i)
            for j in range(i+1, num_vlist):
                if self.inc_matrix[i][j]==1:
                    self.vertex_list[i].adj.append(j)
                    self.vertex_list[j].adj.append(i)
    
    def set_layouts(self):
        self.layouts = Layouts(self)
    
    def init_stuff(self):
        pass
    
    def get_layout(self,layoutnum):
        layout = []
        for i in range(self.num_vertices):
            layout.append(self.vertex_list.pos[layoutnum])
        return layout
    
    def get_subgraph(self):
        pass
    
    def get_positions_from_indexset(self, iset):
    # this grabs a bunch of positions corresponding to the pointers
        vpos = []
        for i in range(len(ptrset)):
            vpos.append(self.vertex_list[ptrset[i]].pos)
        return vpos

    def get_edges_between_indexsets(self, iset0, iset1, color, opacity, line_width):
    # get all of the edges between two disjoint pointer_sets
        edgeset = Edgeset(color, opacity, line_width)
        for i in range(len(iset0)):
            for j in range(len(iset1)):
                if self.inc_matrix[iset0[i]][iset1[j]]==1:
                    edgeset.edges.append(Edge(iset0[i],iset1[j]))
        return edgeset
    
    def get_edges_within_indexset(self, iset, color, opacity, line_width):
    # get all of the edges within a pointer_set
        edgeset = Edgeset(color, opacity, line_width)
        for i in range(len(iset)):
            for j in range(i+1,len(iset)):
                if self.inc_matrix[iset[i]][iset[j]]==1:
                    edgeset.edges.append(Edge(iset[i],iset[j]))
        return edgeset
    
    def induced_subgraph(self, iset):
        subgraph = Graph(len(iset))
        subgraph.vertex_list = []
        for i in iset:
            subgraph.vertex_list.append(self.vertex_list[i])
        subgraph.inc_matrix = []
        for i in range(subgraph.num_vertices):
            row = [0]*subgraph.num_vertices
            subgraph.inc_matrix.append(row)
            if self.inc_matrix[iset[i]][iset[i]] == 1:
                subgraph.inc_matrix[i][i] = 1
        for i in range(subgraph.num_vertices):
            for j in range(i, subgraph.num_vertices):
                if self.inc_matrix[iset[i]][iset[j]] == 1:
                    subgraph.inc_matrix[i][j] = 1
                    subgraph.inc_matrix[j][i] = 1                    
        for i in range(subgraph.num_vertices):
            subgraph.vertex_list[i].adj = []
        subgraph.set_adj_lists()  
        subgraph.indices = [i for i in range(subgraph.num_vertices)]
        subgraph.edges_list = EdgesList(subgraph.get_edges_within_indexset(subgraph.indices, 'black', 0.2, 1), 'all_edges' )  
        subgraph.layouts = copy.deepcopy(self.layouts)
        return subgraph


In [6]:
# layout class

class Layouts:
    def __init__(self, graph):
    # bare minimum
        self.graph = graph
        self.layout_list = [] 
        self.layout_names = []
        self.set_layouts()
        
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def set_layouts(self):
        self.add_layout(self.clockwise_layout(), 'default')
        return self.layout_list, self.layout_names
    
    def add_layout(self,position_list, name):
    # position_list must have num_vertices elements
        for i in range(self.graph.num_vertices):
            self.graph.vertex_list[i].pos.append(position_list[i])
        self.layout_list.append(position_list)
        self.layout_names.append(name)
    
    def clockwise_layout(self):
        vpos_list = []
        angle_increment = 2*np.pi/self.graph.num_vertices    
        for i in range(self.graph.num_vertices):
            vpos_list.append([np.sin(i*angle_increment), np.cos(i*angle_increment), 0])
        return vpos_list

    def get_layouts(self):
        return self.layout_list, self.layout_names

########## helper functions for layouts  #################

    def circle_pos(self, i):
    # gives the ith position around the num_vertices-step unit circle clockwise
        angle_increment = 2*np.pi/self.graph.num_vertices    
        position = [np.sin(i*angle_increment), np.cos(i*angle_increment), 0]
        return position

    def all_circle_positions(self, ptr_list, starting_angle, radius, height):
    # gives equi-spaced positions on a circle of radius and height
        num_ptrs = len(ptr_list)
        angle = -2*np.pi/(num_ptrs)
        pos_list = []
        for i in range(num_ptrs):
            pos_list.append([radius*np.sin(i*angle+starting_angle), radius*np.cos(i*angle+starting_angle), height])
        return pos_list

    def permute_pos(self, vpos_list, permutation):
    # takes a permutation of the form [[x00, x01, ... x0n], ... , [xs0, xs1, ... xsm]]
    # s+1 self-contained permutations, moving however many elements
    # permutes the positions of K as per the permutation
        num_vpos_list = len(vpos_list)
        num_perms = len(permutation)
        #save original positions of vpos_list out
        new_vpos_list = copy.deepcopy(vpos_list)
        for k in range(num_perms):
            num_perm_k = len(permutation[k])
            # move src element position to dest element position, for each move in the permutation
            for i in range(num_perm_k):
                src = permutation[k][i]
                dest = permutation[k][(i+1)%num_perm_k]
                new_vpos_list[dest] = vpos_list[src]
        return new_vpos_list



In [7]:
# the ER Graph class

class ER(Graph):
        
    def set_vertex_list(self, q):
        # generates left-normalized vertices over GF(q)^3 in lexicographic order 
        self.q = q
        vertex_list = [Vertex([0,0,1])]
        for i in range(0,self.q):
            vertex_list.append(Vertex([0,1,i]))
        for i in range(0,self.q):
            for j in range(0,self.q):
                vertex_list.append(Vertex([1,i,j]))
        return vertex_list, len(vertex_list)
    
    def set_incidence_matrix(self):
        # incidence_matrix[i,j] is 1 if vertex_list[i].value is orthogonal to vertex_list[j].value and 0 otherwise
        self.gfq = GFq(self.q)
        num_vertices = len(self.vertex_list)
        inc_matrix = []
        for i in range(num_vertices):
            row = [0]*num_vertices
            inc_matrix.append(row)
        for i in range(num_vertices):
            d = self.gfq.dot_product_3d(self.vertex_list[i].value, self.vertex_list[i].value)
            if d==0:
                inc_matrix[i][i]=1
        for i in range(num_vertices):
            for j in range(i+1,num_vertices):
                d = self.gfq.dot_product_3d(self.vertex_list[i].value, self.vertex_list[j].value)
                if d==0:
                    inc_matrix[i][j]=1
                    inc_matrix[j][i]=1
        return inc_matrix
    
    def set_layouts(self):
        self.layouts = ER_Layouts(self)
    
    def init_stuff(self):
        # the next is specific to ER graphs
        self.L,self.W,self.C,self.V1,self.V2 = self.get_LWCV_subsets_ER()        
        if q<17:
            self.edges_list.list[0].opacity = 0.2
        elif q<27:
            self.edges_list.list[0].opacity = 0.08
        elif q<47:
            self.edges_list.list[0].opacity = 0.02
        elif q<63:
            self.edges_list.list[0].opacity = 0.01
        elif q<129:
            self.edges_list.list[0].opacity = 0.001
        else:
            self.edges_list.list[0].opacity = 0.0002
    
    def get_LWCV_subsets_ER(self):
        L_color = RED
        W_color = DK_RED
        C_color = LT_GREEN
        V1_color = DK_GREEN
        V2_color = LT_BLUE
        L=[]
        W=[]
        # make W
        for i in range(self.num_vertices):
            if self.inc_matrix[i][i]==1:
                W.append(i)
                self.vertex_list[i].color = W_color
        # grab the starter quadric L out of W: the first element of W
        L.append(W[0])
        self.vertex_list[W[0]].color = L_color
        W.pop(0)
        C=[]
        V1=[]
        for i in range(self.num_vertices):
            if (i not in L) and (i not in W) and self.inc_matrix[L[0]][i]==1:
                C.append(i)   
                self.vertex_list[i].color = C_color
        for i in range(self.num_vertices):
            if (i not in L) and (i not in W) and (i not in C):
                for j in W:
                    if self.inc_matrix[i][j]==1:
                        V1.append(i)
                        self.vertex_list[i].color = V1_color
                        break  # only need to be orthogonal to one member of W
        V2 = list(set(range(self.num_vertices)) - set(L+W+C+V1))
        for i in V2:
            self.vertex_list[i].color = V2_color
        return (L, W, C, V1, V2)

    def get_cluster_ER(self, c):
    # get the cluster generated from a given center element
        clusterptrs=[c]
        for i in range(self.num_vertices):
            if self.inc_matrix[c][i]==1 and self.inc_matrix[i][i]==0:
                clusterptrs.append(i)
        return clusterptrs
    
    def get_all_clusters_ER(self):
    # get the cluster generated from a given quadric and center element
        clusters=[]
        for i in range(q):
            cluster=[self.C[i]]
            for j in range(self.num_vertices):
                if self.inc_matrix[self.C[i]][j]==1 and self.inc_matrix[j][j]==0:
                    cluster.append(j)
            clusters.append(cluster)
        return clusters
    
    def set_interesting_edgelists_ER(self):
        cluster_main = self.get_cluster_ER(self.C[0])
        ee = self.get_edges_within_indexset(cluster_main, 'black', 1.0, 3)
        self.edges_list.list.append(self.get_edges_within_indexset(cluster_main, 'black', 1.0, 3))
        self.edges_list.names.append('cluster')

        self.edges_list.list.append(self.get_edges_between_indexsets(cluster_main, self.W+self.L, 'red', 1.0, 3))
        self.edges_list.names.append('cluster_to_quadrics')

        cluster_nbr = self.get_cluster_ER(self.C[len(self.C)-1])
        self.edges_list.list.append(self.get_edges_within_indexset(cluster_nbr, 'black', 1.0, 3)) 
        self.edges_list.names.append('cluster_nbr')

        cluster_nbr_V1 = list(set(cluster_nbr) & set(self.V1))
        self.edges_list.list.append(self.get_edges_between_indexsets(cluster_main, cluster_nbr_V1, LT_GREEN, 1.0, 3)) 
        self.edges_list.names.append('cluster_to_nbrV1')

        cluster_nbr_V2 = list(set(cluster_nbr) & set(self.V2))
        self.edges_list.list.append(self.get_edges_between_indexsets(cluster_main, cluster_nbr_V2, LT_BLUE, 1.0, 3)) 
        self.edges_list.names.append('cluster_to_nbrV2')

        clusters_all=[]
        es = Edgeset('black', 1.0, 2)
        for i in range(len(self.C)):
            clusters_all.append(self.get_cluster_ER(self.C[i]) )
            cluster_es = self.get_edges_within_indexset(clusters_all[i], 'black', 1.0, 3)
            es.concatenate_edgesets(cluster_es.edges)
        self.edges_list.list.append(es)
        self.edges_list.names.append('all_clusters')
        
        return self.edges_list, self.edges_list.names



In [8]:
class ER_Layouts(Layouts):
    
    def set_layouts_ER(self): 
    # in addition to the clockwise default
        self.add_layout(self.good_layout_ER(), 'good') 
        self.add_layout(self.cake_layout_ER(), 'cake')
        return self.layout_list, self.layout_names
            
    def good_layout_ER(self):
    # this method permutes the positions of the vertices in a way that helps to show how clusters are made
        # copy the good layout: cake_layout is based on good_layout
        vpos_list = self.graph.num_vertices*[[0,0,0]]
        floor_q_half = int((self.graph.q-1)/2)
        ceil_q_half = int((self.graph.q+1)/2)
        angle_increment = 2*np.pi/self.graph.num_vertices    
        
        # L: place it (q+1)/2 to the left of the top
        vpos_list[self.graph.L[0]] = self.circle_pos(self.graph.num_vertices-ceil_q_half)
        # C: place these spread over the top of the circle
        for i in range(self.graph.q):
            vpos_list[self.graph.C[i]] = self.circle_pos((i - floor_q_half)%self.graph.num_vertices)
        current_pos = ceil_q_half        
        # W, V1, V2
        count = 1
        for i in range(self.graph.q):      
            # W: interspersed between each "rack" C_i of V1 and V2 points
            if i == floor_q_half:    # put this one opposite L[0]
                cir_pos = ceil_q_half
            else:  # the rest are corresponding to C[i]
                cir_pos = self.graph.num_vertices-ceil_q_half-count*self.graph.q
                count += 1
            for j in range(self.graph.q):
                if self.graph.C[i] in self.graph.vertex_list[self.graph.W[j]].adj:
                    vpos_list[self.graph.W[j]] = self.circle_pos(cir_pos)                
            # V1 and V2: to points corresponding to their cluster centers from C
            V1_Ci=[]
            V2_Ci=[]
            for j in range(len(self.graph.V1)):
                if self.graph.inc_matrix[self.graph.V1[j]][self.graph.C[q-1-i]]==1:
                    V1_Ci.append(self.graph.V1[j])
                if self.graph.inc_matrix[self.graph.V2[j]][self.graph.C[q-1-i]]==1:
                    V2_Ci.append(self.graph.V2[j])
            current_pos += 1
            if (self.graph.q%4 == 3): 
                for j in range(floor_q_half):
                    # move the jth V1_Ci
                    vpos_list[V1_Ci[j]] = self.circle_pos(current_pos)
                    current_pos += 1
                    #  and match it up with the V2 that is orthogonal
                    for k in range(floor_q_half):
                        if V2_Ci[k] in self.graph.vertex_list[V1_Ci[j]].adj:
                            vpos_list[V2_Ci[k]] = self.circle_pos(current_pos)
                            current_pos += 1
                            break # once you've found it, don't need to go on any farther
            else:   # q%4 == 1
                # we'll move through the elts of V1 and V2, popping the pair every time we find a twinned match
                for j in range(int((self.graph.q-1)/4)): 
                    # pick the first V1 and V2 (any one not picked will do), alternate these
                    vpos_list[V1_Ci[0]] = self.circle_pos(current_pos)
                    current_pos += 1
                    #pick twins of V1_temp[0]
                    for k in range(1,floor_q_half):
                        if V1_Ci[k] in self.graph.vertex_list[V1_Ci[0]].adj:
                            vpos_list[V1_Ci[k]] = self.circle_pos(current_pos)
                            V1_Ci.pop(k)
                            V1_Ci.pop(0)
                            current_pos += 1
                            break # once you've found it, don't need to go on any farther
                    vpos_list[V2_Ci[0]] = self.circle_pos(current_pos)
                    current_pos += 1
                    #pick twins of V2_temp[0]
                    for k in range(1,floor_q_half):
                        if V2_Ci[k] in self.graph.vertex_list[V2_Ci[0]].adj:
                            vpos_list[V2_Ci[k]] = self.circle_pos(current_pos)
                            V2_Ci.pop(k)
                            V2_Ci.pop(0)
                            current_pos += 1
                            break # once you've found it, don't need to go on any farther               
        return vpos_list

    def cake_layout_ER(self):
    # copy the good layout: cake_layout is based on good_layout
        vpos_list = []
        for i in range(self.graph.num_vertices):
            vpos_list.append(copy.deepcopy(self.graph.vertex_list[i].pos[1]) )      
    # L,W: move away from the viewer and into the center, starting_angle=0, height=-1
        LW = self.graph.L+self.graph.W
        radius_W = .25
        LW_vpos_list = self.all_circle_positions(LW, 0, radius_W, -1.0)
        vpos_list[self.graph.L[0]] = LW_vpos_list[0]
        for i in range(self.graph.q):
            vpos_list[self.graph.W[i]] = LW_vpos_list[i+1]
    # C: Move into the center, starting_angle=-np.pi/q, height=-0
        radius_C= 1.5*radius_W
        C_vpos_list = self.all_circle_positions(self.graph.C, -np.pi/q, radius_C, 0)
        for i in range(self.graph.q):
            vpos_list[self.graph.C[i]] = C_vpos_list[i]
    # V1: leave them where they are in the good layout
    # V2: Move the toward the viewer at positive z
        for i in range(len(self.graph.V2)):        
            vpos_list[self.graph.V2[i]][2] += 1.0
    # This permutation on the quadrics is used in the paper (Fig. 7) to cleanly show quadrics connections for q=7        
        if self.graph.q == 7:
            for i in range(self.graph.q+1):
                LW_vpos_list[i] = vpos_list[LW[i]]
            new_LW_vpos_list = self.permute_pos(LW_vpos_list, [[0,1,2,4,6,7]])
            for i in range(self.graph.q+1):
                vpos_list[LW[i]] = new_LW_vpos_list[i]
        return vpos_list


In [9]:
# random_val = .2

# class RandGraph(Graph):
    
#     def set_vertex_list(self, num_vert):
#         vertex_list = []
#         for i in range(0,num_vert):
#             vertex_list.append(Vertex(i))
#         return vertex_list, num_vert

#     def set_incidence_matrix(self):
#         inc_matrix = []
#         for i in range(self.num_vertices):
#             row = [0]*self.num_vertices
#             inc_matrix.append(row)
#             if np.random.rand() < random_val:
#                 inc_matrix[i][i] = 1                
#         for i in range(self.num_vertices):
#             for j in range(i+1, self.num_vertices):
#                 if np.random.rand() < random_val:
#                     inc_matrix[i][j] = 1                
#                     inc_matrix[j][i] = 1                
#         return inc_matrix


# num_vertices = [10,13,25]

# for num_vert in num_vertices:
#     rand_graph = RandGraph(num_vert)
#     visualize_graph(rand_graph, 'default', 
#                     ['all_edges'], 
#                      'xy', [0,0,0], False,
#                      "rand_test", 
#                     'rand('+str(num_vert)+'): random graph, default layout')
    
    

In [10]:
#####    these are the prime powers q to be run         ##### 
#####    this algorithm is for ODD PRIME POWERS only    ##### 

# Here are some smaller primes
#prime_power_list = [3,5,7,9,11,13,17,19,23,25,27,29,31,37,41,43,47,49,53,59,61]

# These primes are of interest.
# Graphs for these primes support [556,993,2257,3783,16257] routers that have radixes [24,32,48,62,128]. 
# May want to change the opacity on edges and maybe the line width in the viz.
#prime_power_list = [23,31,47,61,127]  

#prime_power_list = [3,7]  # used for the paper viz

prime_power_list = [7,9]

for q in prime_power_list:
    ER_Graph = ER(q)
    layout_list, layout_names = ER_Graph.layouts.set_layouts_ER()
    edgelists,edgelist_names = ER_Graph.set_interesting_edgelists_ER()
    
    ER_Subgraph = ER_Graph.induced_subgraph(ER_Graph.V1)
    
    visualize_graph(ER_Subgraph, 'good', 
                    ['all_edges'], 
                     'xy', [0,0,0], False,
                     "test", 
                    'GF('+str(q)+'): a subgraph, good layout')
           
# have a look at the basic layouts

# visualize_graph(graph, layout, edge_displaylist, camera_pos, camera_rot, show_labels, fname, caption)
# layouts: ['default', 'good', 'cake']
# edgelists: ['all_edges', 'cluster', 'cluster_to_quadrics', 'cluster_nbr', 'cluster_to_nbrV1', 'cluster_to_nbrV2', 'all_clusters']

    visualize_graph(ER_Graph, 'default', 
                    ['all_edges'], 
                    'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_lex", 
                    'GF('+str(q)+'): Lexicographic layout')
    visualize_graph(ER_Graph, 'good', 
                    ['all_edges'], 
                    'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_good", 
                    'GF('+str(q)+'): Good layout')
#     visualize_graph(ER_Graph, 'cake', 
#                     ['all_edges'], 
#                     'zx', [0,0,20], False,
#                      "./ER_graphs/ER" + str(q) + "_cake_all", 
#                     'GF('+str(q)+'): Layer cake, all edges')    
# #   look at one cluster, cake layout
#     visualize_graph(ER_Graph, 'cake', 
#                     ['all_edges', 'cluster'],
#                     'zx', [0,0,20], False,
#                      "./ER_graphs/ER" + str(q) + "_cake_1cluster", 
#                     'GF('+str(q)+'): Layer cake, one cluster')    
# #   look at the same cluster, plus its connections to the quadrics, cake layout
#     visualize_graph(ER_Graph, 'cake', 
#                     ['all_edges', 'cluster', 'cluster_to_quadrics'],
#                     'zx', [0,0,20], False,
#                      "./ER_graphs/ER" + str(q) + "_cake_1cluster_quads", 
#                     'GF('+str(q)+'): Layer cake, one cluster plus quadrics')    
#   look at the same cluster, plus its connections to the quadrics and to a neighbor cluster, cake layout
    visualize_graph(ER_Graph, 'cake', 
                    ['all_edges', 'cluster', 'cluster_to_quadrics', 'cluster_nbr', 'cluster_to_nbrV1', 'cluster_to_nbrV2'],
                    'zx', [0,0,20], False,
                     "./ER_graphs/ER" + str(q) + "_cake_2clusters_edges", 
                    'GF('+str(q)+'): Layer cake, two clusters plus external edges')   
#   look at all clusters from overhead, cake layout
    visualize_graph(ER_Graph, 'cake', 
                    ['all_edges', 'all_clusters'], 
                    'xy', [0,0,0], False,
                     "./ER_graphs/ER" + str(q) + "_cake_allclusters_overhead", 
                    'GF('+str(q)+'): Layer cake, all clusters, overhead')
    


GF(7): a subgraph, good layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(7): Lexicographic layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(7): Good layout


...

ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(9): Layer cake, two clusters plus external edges


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF(9): Layer cake, all clusters, overhead


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)